In [163]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from ast import literal_eval


In [164]:
var_dict = {
    'T':'Time_Spend_Company',
    'D':'Department',
    'NP':'Number_Projects',
    'P':'Promotion_Last_5_Years',
    'L':'Left',
    'H':'Average_Montly_Hours',
    'WA':'Work_Accident',
    'LE':'Last_Evaluation',
    'S':'Salary',
    'SL':'Satisfaction_Level'
}

def item2dict(item):
    ## splitto la stringa
    item_split = item.split('_')
    # definisco la lista variabile,valore
    item_list = {var_dict[item_split[1]] : item_split[0]}
    return item_list


In [165]:
def df2df_set(df):
    """funzione che converte 
    le stringhe relative alle regole in 
    liste di dizionari """

    df_new = df.copy()

    df_new = df_new.drop(['Antecedent','Consequent'],axis=1)

    Antecedent=[]
    for i,item in enumerate(df.Antecedent):
        Antecedent.append(set(literal_eval(item)))

    df_new=df_new.assign(Antecedent=Antecedent)

    Consequent=[]
    for i,item in enumerate(df.Consequent):
        Consequent.append(item2dict(item))

    df_new=df_new.assign(Consequent=Consequent)
        
    return df_new        


# Analisi regole Per LEFT

In [248]:
df = pd.read_csv("../../data/rules/rules_supp5_conf50.csv")
df.shape
#df_filtered = df.query('Supp>0.2') # perchè non è maggiore di 20???
#df_filtered = df_filtered.query('Lift>1')
df_left = df.query(" Consequent=='Y_L' and Lift>1")

df_left.to_csv("../../code/rules/df_left_all.csv")
df_left.describe()

# converto in dataset di set
df_left_set=df2df_set(df_left)
df_left_set.head()

# aggiungo la lunghezza della parte antecedente
N_items=[]
ant = df_left_set.Antecedent
N_items = []
for i,ant in enumerate(df_left_set.Antecedent):
    N_items.append(len(df_left_set.Antecedent.iloc[i]))


df_left_set['N_items']=N_items
df_left_set.head()


     Supp   Conf   Lift                     Antecedent       Consequent  \
79  0.050  0.774  3.252  {intensive_H, N_P, N_WA, 5_T}  {u'Left': u'Y'}   
82  0.050  0.772  3.243       {intensive_H, N_WA, 5_T}  {u'Left': u'Y'}   
85  0.053  0.741  3.111        {intensive_H, N_P, 5_T}  {u'Left': u'Y'}   
87  0.053  0.737  3.094             {intensive_H, 5_T}  {u'Left': u'Y'}   
90  0.053  0.615  2.583               {N_P, N_WA, 5_T}  {u'Left': u'Y'}   

    N_items  
79        4  
82        3  
85        3  
87        2  
90        3  

# Inizio pruning delle regole

In [249]:
# pre-filtering
df_left_filtered=df_left_set.query('Conf>0.5')
df_left_filtered.describe()

             Supp        Conf        Lift     N_items
count  264.000000  264.000000  264.000000  264.000000
mean     0.070557    0.811826    3.409856    4.121212
std      0.020260    0.131242    0.551197    1.293625
min      0.050000    0.507000    2.131000    1.000000
25%      0.053000    0.720750    3.028000    3.000000
50%      0.061500    0.852500    3.580500    4.000000
75%      0.086000    0.924250    3.882500    5.000000
max      0.154000    0.977000    4.103000    8.000000

In [250]:
# rimuovo quelli le regole contenenti quelli che non hanno avuto incidenti
data= df_left_filtered.copy()

indici_remove=[]
for i in range(data.shape[0]):
    if set(['N_WA']).issubset( data.iloc[i].Antecedent ):
        indici_remove.append(i)
#        print i
        #         print data.iloc[i].Antecedent
print len(indici_remove)
data.drop(data.index[indici_remove],inplace=True)
data.describe()
data_clean=data
data_clean.head()
del(data)

#data_clean['Supp_round'] = data_clean.Supp.round(2)
#data_clean.head()

136


In [251]:
data=data_clean.query('Conf>0.85')
data.describe()
#data.sort_values(by='Supp',ascending=False).Supp.round(2)

            Supp       Conf       Lift    N_items
count  62.000000  62.000000  62.000000  62.000000
mean    0.069855   0.917258   3.852677   4.403226
std     0.017583   0.035456   0.149375   0.999339
min     0.053000   0.857000   3.598000   2.000000
25%     0.053000   0.884000   3.714250   4.000000
50%     0.062000   0.924500   3.883000   4.000000
75%     0.086000   0.946500   3.976000   5.000000
max     0.101000   0.975000   4.097000   7.000000

In [252]:
data=data.sort_values(by='N_items',ascending=False)
data_pruned=data.copy()
data.head()

       Supp   Conf   Lift                                         Antecedent  \
736   0.053  0.975  4.097  {low_S, 3_T, N_P, 2_NP, medium_SL, insufficien...   
845   0.053  0.954  4.008  {low_S, medium_SL, 2_NP, N_P, insufficient_LE,...   
1117  0.061  0.942  3.955     {low_S, 3_T, medium_SL, 2_NP, N_P, standard_H}   
941   0.053  0.961  4.037  {low_S, 3_T, 2_NP, N_P, insufficient_LE, stand...   
743   0.053  0.975  4.097  {low_S, 3_T, 2_NP, medium_SL, insufficient_LE,...   

           Consequent  N_items  
736   {u'Left': u'Y'}        7  
845   {u'Left': u'Y'}        6  
1117  {u'Left': u'Y'}        6  
941   {u'Left': u'Y'}        6  
743   {u'Left': u'Y'}        6  

In [253]:
#print data_pruned.head()
# ordino

# prendo la prima
row=0
while row<data_pruned.shape[0] :
    rule = data.iloc[row].Antecedent
    supp_rule = data.iloc[row].Supp
    
    #print row
    indici_remove=[]
    for i in range((row+1),(data_pruned.shape[0])):
        successiva = data_pruned.iloc[i]
        if rule.issuperset(successiva.Antecedent) and (round(supp_rule,2)==round(successiva.Supp,2)):
            print round(supp_rule,2)
            print round(successiva.Supp,2)
            indici_remove.append(i)
            #print supp_rule
            #print successiva.Supp
    data_pruned.drop(data_pruned.index[[indici_remove]],inplace=True)
    del(indici_remove)
    
    row+=1



0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.05
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.06
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.09
0.05
0.05
0.05
0.05
0.05
0.05


In [254]:
data_pruned.describe()
#data_pruned.sort_values(by='Supp',ascending=False)


           Supp       Conf       Lift    N_items
count  12.00000  12.000000  12.000000  12.000000
mean    0.08575   0.921250   3.869667   4.750000
std     0.01907   0.037407   0.157462   1.215431
min     0.05300   0.858000   3.603000   3.000000
25%     0.07975   0.894500   3.758250   4.000000
50%     0.09300   0.924500   3.883000   5.000000
75%     0.10025   0.951250   3.995750   5.250000
max     0.10100   0.975000   4.097000   7.000000

In [255]:
data_pruned.to_csv("../../data/rules/pruned_rules_superset.csv")                            

In [ ]:
FINE ##################################

In [214]:
#df_left[data_]
data_pruned.index
#data_pruned.to_csv("../../data/best_rules.csv")

Int64Index([736, 1117, 793, 140, 990, 1166, 818, 1171, 1187, 1331, 1335, 1191], dtype='int64')

In [195]:
data_pruned.index

Int64Index([736, 1117, 793, 140, 990, 1166, 818, 1331, 1187, 1171, 1335, 1191], dtype='int64')

In [196]:
data_best_rules=df_left.loc[data_pruned.index]
data_pruned.to_csv("../../data/rules/best_rules.csv")



In [197]:
def prune_rules(data):

    data=data.sort_values(by='N_items')
    data_pruned=data.copy()

    #print data_pruned.head()
    # ordino

    # prendo la prima
    row=0
    while row<data_pruned.shape[0] :
        rule = data.iloc[row].Antecedent
        supp_rule = data.iloc[row].Supp

        #print row
        indici_remove=[]
        for i in range((row+1),(data_pruned.shape[0])):
            successiva = data_pruned.iloc[i]
            if rule.issuperset(successiva.Antecedent) and (supp_rule==successiva.Supp):
                indici_remove.append(i)
                #print supp_rule
                #print successiva.Supp
        data_pruned.drop(data_pruned.index[[indici_remove]],inplace=True)
        del(indici_remove)

        row+=1
    return data_pruned



In [198]:
data=data_clean.query('Conf>0.95')
data.sort_values(by='N_items').describe()

            Supp       Conf       Lift    N_items
count  14.000000  14.000000  14.000000  14.000000
mean    0.062429   0.962143   4.042143   5.357143
std     0.015471   0.008556   0.036027   0.841897
min     0.053000   0.952000   3.999000   4.000000
25%     0.053000   0.954750   4.011750   5.000000
50%     0.053000   0.961000   4.037500   5.000000
75%     0.077750   0.970500   4.076750   6.000000
max     0.086000   0.975000   4.097000   7.000000

In [83]:
data_pruned = prune_rules(data)
data_pruned.describe()
data_pruned



      Supp   Conf   Lift                                         Antecedent  \
823  0.086  0.957  4.020            {2_NP, medium_SL, 3_T, insufficient_LE}   
971  0.053  0.952  3.999                {low_S, 2_NP, 3_T, insufficient_LE}   
771  0.053  0.973  4.087     {low_S, 2_NP, medium_SL, 3_T, insufficient_LE}   
799  0.086  0.962  4.041  {standard_H, 2_NP, medium_SL, 3_T, insufficien...   
818  0.086  0.958  4.025       {N_P, 2_NP, medium_SL, 3_T, insufficient_LE}   
851  0.053  0.954  4.009  {low_S, 2_NP, medium_SL, standard_H, insuffici...   
947  0.053  0.961  4.038    {low_S, 2_NP, standard_H, 3_T, insufficient_LE}   
966  0.053  0.952  3.999           {low_S, 2_NP, N_P, 3_T, insufficient_LE}   
743  0.053  0.975  4.097  {low_S, 3_T, 2_NP, medium_SL, insufficient_LE,...   
765  0.053  0.973  4.087  {low_S, 3_T, medium_SL, 2_NP, N_P, insufficien...   
793  0.086  0.963  4.046  {3_T, medium_SL, 2_NP, N_P, insufficient_LE, s...   
845  0.053  0.954  4.008  {low_S, medium_SL, 2_NP, N

In [86]:
data_pruned

      Supp   Conf   Lift                                         Antecedent  \
823  0.086  0.957  4.020            {2_NP, medium_SL, 3_T, insufficient_LE}   
971  0.053  0.952  3.999                {low_S, 2_NP, 3_T, insufficient_LE}   
771  0.053  0.973  4.087     {low_S, 2_NP, medium_SL, 3_T, insufficient_LE}   
799  0.086  0.962  4.041  {standard_H, 2_NP, medium_SL, 3_T, insufficien...   
818  0.086  0.958  4.025       {N_P, 2_NP, medium_SL, 3_T, insufficient_LE}   
851  0.053  0.954  4.009  {low_S, 2_NP, medium_SL, standard_H, insuffici...   
947  0.053  0.961  4.038    {low_S, 2_NP, standard_H, 3_T, insufficient_LE}   
966  0.053  0.952  3.999           {low_S, 2_NP, N_P, 3_T, insufficient_LE}   
743  0.053  0.975  4.097  {low_S, 3_T, 2_NP, medium_SL, insufficient_LE,...   
765  0.053  0.973  4.087  {low_S, 3_T, medium_SL, 2_NP, N_P, insufficien...   
793  0.086  0.963  4.046  {3_T, medium_SL, 2_NP, N_P, insufficient_LE, s...   
845  0.053  0.954  4.008  {low_S, medium_SL, 2_NP, N

In [162]:
data_pruned.to_csv("../../data/rules/pruned_rules_superset.csv")